In [1]:
from __future__ import print_function
import pandas as pd
import gspread_pandas
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from tqdm import tqdm
from time import sleep
import yaml

In [7]:
config=gspread_pandas.conf.get_config(conf_dir='/home/dev-cory/learning_english/credential/',file_name='client_secret_english_crawl.json')
#gspread_pandas.conf.get_config
gspread_pandas.conf.get_config
with open('./credential/learning_english.yaml','r') as ymlfile:
    cfg = yaml.load(ymlfile,Loader=yaml.BaseLoader)
app_id = cfg['OxfordAPI']['app_id']
app_key = cfg['OxfordAPI']['app_key']
language = 'en'

In [8]:
spread = gspread_pandas.Spread('Oxford vocabulary',config=config)

In [9]:
df=spread.sheet_to_df(sheet='vocab')

In [10]:
df

,Word,Importance,Meaning in Japanese,Meaning in English,Example,Origin,Pronunciation,Mp3 - pronunciation
ru,,,,,,,,
1,retaliate,3,仕返しする,1: make an attack in return for a similar atta...,,early 17th century: from Latin retaliat-‘retur...,rɪˈtalɪeɪt,http://audio.oxforddictionaries.com/en/mp3/ret...
2,detain,3,拘束する,keep (someone) from proceeding by holding them...,,late Middle English (in the sense ‘be afflicte...,dɪˈteɪn,http://audio.oxforddictionaries.com/en/mp3/det...
3,resilient,3,はね返る、弾力のある、たちまち元気を回復する、快活な、溌刺(はつらつ)とした,(of a person or animal) able to withstand or r...,,mid 17th century: from Latin resilient-‘leapin...,rɪˈzɪlɪənt,http://audio.oxforddictionaries.com/en/mp3/res...
4,lineage,4,血統、系統、家柄,direct descent from an ancestor; ancestry or p...,,"Middle English: from Old French lignage, from ...",ˈlɪnɪɪdʒ,http://audio.oxforddictionaries.com/en/mp3/lin...
5,migration,1,移住、転住、渡り、移住者群、移動する動物の群,1: seasonal movement of animals from one regio...,,,mʌɪˈɡreɪʃ(ə)n,http://audio.oxforddictionaries.com/en/mp3/mig...
...,...,...,...,...,...,...,...,...
1386,epicentre,,,,Twenty kilometres up from the epicentre and a ...,,,
1387,frazzle,,,,"In the first, frazzled workers shovel huge pil...",,,
1388,fiery,,,,"In the first, frazzled workers shovel huge pil...",,,


In [11]:
 df.loc['4','Meaning in Japanese']

'血統、系統、家柄'

In [12]:
def weblio_crawler(word_id):
    url ='https://ejje.weblio.jp/content/'+word_id
    try:
        html_page = urlopen(url)
        soup = BeautifulSoup(html_page)
        text=soup.find('td',attrs={"class", "content-explanation ej"})
        answer=text.contents[0]
    except:
        answer=''
    return answer

In [13]:
def Oxford_definition_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        x=1
        answer=''
        for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
            answer=answer +str(x)+': '+str(i["definitions"][0])+'  '
            x+=1
            sleep(5)
    except:
        answer=''
    return answer

In [14]:
def Oxford_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        sleep(5)
    except Exception as e:
        print(e)
        info=''
    return info

In [15]:
def Oxford_example_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        x=1
        answer=''
        for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
            answer=answer +str(x)+': '+str(i["examples"][0]["text"])+'  '
            x+=1
            sleep(5)
    except:
        answer=''
    return answer

In [16]:
def Oxford_info_processor(info,target):
    try:
        answer=''
        if target=='meaning':
            x=1
            for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
                answer=answer +str(x)+': '+str(i["definitions"][0])+' \r\n'
                x+=1       
        elif target=='phonic':
            answer=info["results"][0]['lexicalEntries'][0]["pronunciations"][0]["phoneticSpelling"]
        elif target=='example':
            x=1
            for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
                answer=answer +str(x)+': '+str(i["examples"][0]["text"])+' \r\n'
                x+=1
        elif target=='mp3':
            answer=info["results"][0]['lexicalEntries'][0]["pronunciations"][0]["audioFile"]
        elif target=='origin':
            answer=info["results"][0]['lexicalEntries'][0]['entries'][0]['etymologies'][0]
    except Exception as e:
        print(e)
        answer=''
    return answer

In [17]:
for index in tqdm(df.index):
    word_id=df.loc[index,'Word']
    word_id=word_id.strip(' ')
    word_id=word_id.lower()
    if df.loc[index,'Meaning in Japanese']=='':
        df.loc[index,'Meaning in Japanese']=weblio_crawler(word_id)
    if df.loc[index,'Meaning in English']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Meaning in English']=Oxford_info_processor(info,'meaning')
        #print(Oxford_info_processor(info,'meaning'))
        if df.loc[index,'Example']=='':
            df.loc[index,'Example']=Oxford_info_processor(info,'example')
        if df.loc[index,'Origin']=='':
            df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
            df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
            #print(Oxford_info_processor(info,'phonic'))
            df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    '''
    if df.loc[index,'Example']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Example']=Oxford_info_processor(info,'example')
        if df.loc[index,'Origin']=='':
            df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
            df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
            df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    if df.loc[index,'Origin']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
        df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
        df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    '''
    
        

  2%|▏         | 32/1378 [00:05<04:00,  5.60it/s]

'definitions'
'examples'


  3%|▎         | 41/1378 [00:11<07:03,  3.16it/s]

'definitions'
'examples'
'etymologies'


 10%|▉         | 135/1378 [00:17<04:58,  4.17it/s]

...

100%|█████████▉| 1373/1378 [24:26<00:39,  7.95s/it]

'etymologies'


100%|█████████▉| 1376/1378 [24:49<00:15,  7.77s/it]

'etymologies'


100%|██████████| 1378/1378 [25:06<00:00,  1.09s/it]


In [18]:
spread.df_to_sheet(df, index=True, sheet='result', start='A1', replace=True)

In [ ]:
df